In [14]:
# guarda como scrape_tolima_selenium_detalle.py
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from urllib.parse import urljoin
import pandas as pd
import time

BASE = "https://www.tolima.travel"
LISTING_TEMPLATE = BASE + "/prestadores-turisticos?page={}"

def init_driver(headless=False):
    opts = webdriver.ChromeOptions()
    if headless:
        opts.add_argument("--headless=new")
    opts.add_argument("--start-maximized")
    # puedes añadir opciones extra si las necesitas
    return webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=opts)

def extract_description_and_hours(driver, wait):
    """Extrae descripción y horarios desde la página de detalle ya cargada."""
    descripcion = ""
    horarios = ""
    try:
        # esperar a que el body esté presente
        wait.until(EC.presence_of_element_located((By.TAG_NAME, "body")))
        time.sleep(0.5)

        # 1) Buscar Descripción: h2 "Descripción" -> primer p.text-description después de ese h2
        xpath_desc = ("//h2[normalize-space(.)='Descripción' or normalize-space(.)='Descripcion']"
                      "//following::p[contains(@class,'text-description')][1]")
        elems = driver.find_elements(By.XPATH, xpath_desc)
        if elems:
            descripcion = elems[0].text.strip()
        else:
            # fallback: tomar el p.text-description más largo de la página (muy frecuente en este site)
            p_all = driver.find_elements(By.CSS_SELECTOR, "p.text-description")
            if p_all:
                descripcion = max(p_all, key=lambda e: len(e.text or "")).text.strip()

        # 2) Buscar Horarios: h2 "Horarios" -> primer p.text-description después de ese h2
        xpath_hor = ("//h2[normalize-space(.)='Horarios' or contains(translate(normalize-space(.),'ÁÉÍÓÚ','AEIOU'),'HORARIOS') ]"
                     "//following::p[contains(@class,'text-description')][1]")
        elems_h = driver.find_elements(By.XPATH, xpath_hor)
        if elems_h:
            horarios = elems_h[0].text.strip()
        else:
            # fallback: buscar cualquier p.text-description que contenga palabras clave o pattern de hora
            p_all = driver.find_elements(By.CSS_SELECTOR, "p.text-description")
            for p in p_all:
                txt = (p.text or "").strip()
                low = txt.lower()
                if "horario" in low or "atención" in low or "atencion" in low or ":" in txt:
                    horarios = txt
                    break

    except Exception as e:
        print("⚠️ Error extrayendo detalle:", e)

    return descripcion, horarios

def scrape_tolima(driver, headless=False):
    wait = WebDriverWait(driver, 15)
    results = []

    for page in range(1, 7):
        url = LISTING_TEMPLATE.format(page)
        print(f"\n== Página {page}: {url}")
        driver.get(url)

        # esperar que el contenedor de cards exista
        try:
            wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "div.tourist-site-content__card-content")))
        except:
            print("No cargó correctamente el listado (timeout). Intento de todas formas.")
        time.sleep(1)

        # tomar todos los cards visibles (app-tt-card o selector alternativo)
        cards = driver.find_elements(By.CSS_SELECTOR, "div.tourist-site-content__card-content app-tt-card, app-tt-card.tt-card, div.card-list app-tt-card")
        print(f" → Encontradas {len(cards)} tarjetas en la página {page}")

        # recolectar metadata y hrefs primero (para evitar stale elements)
        infos = []
        for c in cards:
            try:
                nombre = ""
                try:
                    nombre = c.find_element(By.CSS_SELECTOR, "h1.card-body__title").text.strip()
                except:
                    pass

                ubicacion = ""
                categoria = ""
                for p in c.find_elements(By.TAG_NAME, "p"):
                    txt = (p.text or "").strip()
                    if "Ubicación" in txt or "Ubicaci" in txt:
                        partes = txt.split(":", 1)
                        ubicacion = partes[1].strip() if len(partes) > 1 else txt
                    if "Categoría" in txt or "Categoria" in txt:
                        partes = txt.split(":", 1)
                        categoria = partes[1].strip() if len(partes) > 1 else txt

                href = ""
                # el enlace "Ver más" es <a class="see-more" href="...">
                try:
                    a = c.find_element(By.CSS_SELECTOR, "a.see-more")
                    href = a.get_attribute("href") or ""
                except:
                    # fallback: cualquier <a> dentro del card con href relativo a prestadores-turisticos
                    for a in c.find_elements(By.TAG_NAME, "a"):
                        h = a.get_attribute("href") or ""
                        if h and ("prestadores-turisticos" in h):
                            href = h
                            break

                href = urljoin(BASE, href) if href else ""
                infos.append({"nombre": nombre, "ubicacion": ubicacion, "categoria": categoria, "url": href})
            except Exception as e:
                print(" ⚠️ error leyendo card (se omite):", e)
                continue

        # Abrir cada detalle en pestaña nueva y extraer descripción/horarios
        for info in infos:
            desc = ""
            hrs = ""
            url_d = info["url"]
            print("   - Procesando:", info["nombre"], url_d)
            if url_d:
                # abrir nueva pestaña con la URL
                driver.execute_script("window.open(arguments[0]);", url_d)
                driver.switch_to.window(driver.window_handles[-1])
                # extraer
                desc, hrs = extract_description_and_hours(driver, wait)
                # cerrar pestaña y volver
                driver.close()
                driver.switch_to.window(driver.window_handles[0])
                time.sleep(0.3)

            results.append({
                "Nombre": info["nombre"],
                "Ubicación": info["ubicacion"],
                "Categoría": info["categoria"],
                "Descripción": desc,
                "Horarios": hrs,
                "URL": url_d,
                "Page": page
            })

    return results

if __name__ == "__main__":
    driver = init_driver(headless=False)  # pon True para headless
    try:
        registros = scrape_tolima(driver)
    finally:
        driver.quit()

    df = pd.DataFrame(registros)
    df.to_csv("tolima_prestadores_detalle.csv", index=False, encoding="utf-8-sig")
    print(f"\n✅ Guardados {len(df)} registros en tolima_prestadores_detalle.csv")



== Página 1: https://www.tolima.travel/prestadores-turisticos?page=1
 → Encontradas 12 tarjetas en la página 1
   - Procesando: Villas Kairos Deluxe https://www.tolima.travel/prestadores-turisticos/villas-kairos-deluxe
   - Procesando: Hospedaje Casa el Cedral https://www.tolima.travel/prestadores-turisticos/hospedaje-casa-el-cedral
   - Procesando: Hotel Kynzha https://www.tolima.travel/prestadores-turisticos/hotel-kynsha
   - Procesando: Colombia Bonita https://www.tolima.travel/prestadores-turisticos/colombia-bonita
   - Procesando: Hotel Posada Casa del Río https://www.tolima.travel/prestadores-turisticos/hotel-posada-casa-del-rio
   - Procesando: Villas del Paraíso https://www.tolima.travel/prestadores-turisticos/villas-del-paraiso
   - Procesando: Finca Villa Magaly https://www.tolima.travel/prestadores-turisticos/finca-villa-magaly
   - Procesando: Finca Turística Las Cabañas de Rovira https://www.tolima.travel/prestadores-turisticos/finca-turistica-las-cabanas-de-rovira
   - P